# Code Original



In [22]:
import tensorflow as tf
import numpy as np

class TBHCNN(tf.keras.Model):
    def __init__(self):
        super(TBHCNN, self).__init__()
        self.H_size_added = 0
        self.HR = None
        self.R = None
        self.R_without_opposite = None
        self.K = None
        self.references = None
        self.wholebandstructure = None
        self.reproduces = None

    def read_training_set(self, references, kvectors):
        numk, numb = references.shape
        self.numb = numb
        self.numk = numk
        self.H_size_init = numb
        self.H_size = numb
        self.references = tf.constant(references, dtype=tf.float64)
        self.K = tf.constant(kvectors, dtype=tf.complex64)
    
    def define_TB_representation(self, rvectors_without_opposite):
        len_rvector = rvectors_without_opposite.shape[0]
        numr = 2 * len_rvector - 1
        self.R = np.array([])
        
        for i in rvectors_without_opposite:
            self.R = np.append(self.R, i)
            if np.sum(np.abs(i)) != 0:
                self.R = np.append(self.R, -1*i)
                
        self.numr = numr        
        self.R = self.R.reshape(-1, 3)
        self.R = tf.cast(self.R, dtype=tf.complex64)
        self.R_without_opposite = rvectors_without_opposite

    def reinitialize(self):
        self.HR = []
        self.H_size = self.H_size_init + self.H_size_added
        
        for i in self.R_without_opposite:
            H_tmp = tf.Variable(tf.random.truncated_normal([self.H_size, self.H_size], mean=0.0, stddev=1.0, dtype=tf.float64))
            H_tmp = tf.cast(H_tmp, dtype=tf.complex64)
            
            if np.sum(np.abs(i)) != 0:
                self.HR.append(H_tmp)
                self.HR.append(tf.transpose(H_tmp))
            else:
                self.HR.append(H_tmp + tf.transpose(H_tmp))
                
        self.HR = tf.stack(self.HR)
    
    def call(self, inputs):
        # Assuming 'inputs' will be the k-vectors
        return self.compute_bands(inputs)


    def compute_bands(self,k_vectors):
        reproduces = tf.zeros([self.numk, self.H_size], dtype=tf.float64)
        for i in range(self.numk):
            HK = tf.zeros([self.H_size, self.H_size], dtype=tf.complex64)
            for j in range(self.numr):
                HK += tf.scalar_mul(tf.exp(1j * tf.reduce_sum(self.K[i] * self.R[j])), self.HR[j])
                
            e = tf.linalg.eigvalsh(HK)
            e = tf.cast(e, dtype=tf.float64)
            e = tf.reshape(e, [1, self.H_size])
            reproduces += tf.tensor_scatter_nd_update(reproduces, [[i]], e)
         
        self.wholebandstructure = reproduces
        reproduces = reproduces[:, int(self.H_size_added/2):int(self.H_size_added/2) + self.numb]
        self.reproduces = reproduces
        
        return reproduces

In [23]:


# we take the example of the 13-atom-wide InSe

# Define the tight-binding representation by selecting the real-space Hamiltonian considered
"""
Here, to ensure that the Hamiltonians with the opposite lattice vectors are always taken into consideration simultaneously, and they tranpose each other
only one of each pair of the opposite lattice vectors should be included in the rvectors array, the other one will be handled automatically
e.g., for this system, both np.array([[0,0,0],[0,0,1],]) and np.array([[0,0,0],[0,0,-1],]) represent the lattice vector set np.array([[0,0,0],[0,0,1],[0,0,-1]])
and in the end we shall get the 3 real-space Hamiltonian matrices we actually used to build the tight-binding model
"""
rvectors_without_opposite = np.array([[0,0,0],[0,0,1],], dtype=np.int32) # in units of[a, b, c] (a, b, and c are the real-space basis vectors; [l, n, m] means the lattice vector l*a+n*b+m*c)

# Load data
references = np.load("./data/input/InSe Nanoribbon/InSe-references.npy")
kvectors = np.load("./data/input/InSe Nanoribbon/InSe-kpoints.npy")

# Hyperparameters
learning_rate = 0.001
loss_threshold = 1e-5
max_training_steps = 10000
basis_added_step = 2

def train_step(model, inputs, outputs, optimizer):
    with tf.GradientTape() as tape:
        predictions = model(inputs, training=True)  # Ensure you call the model on the inputs to get the predictions
        loss = tf.reduce_mean(tf.square(predictions - outputs))
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    return loss  # Make sure to return the loss from the function

def fitting(model, optimizer, loss_threshold, max_train_steps, references, kvectors):
    train_steps = 0
    loss = 0
    for step in range(max_train_steps):
        loss = train_step(model, kvectors, references, optimizer)
        if step % 1000 == 0 or step == max_train_steps - 1:
            print(f"Step {step}, Loss: {loss.numpy():.8f}")
        if loss < loss_threshold:
            break
    return loss < loss_threshold

def main():
    model = TBHCNN()
    model.read_training_set(references, kvectors)
    model.define_TB_representation(np.array([[0,0,0],[0,0,1]], dtype=np.int32))
    optimizer = tf.keras.optimizers.Adam(learning_rate)

    finished = fitting(model, optimizer, loss_threshold, max_training_steps, references, kvectors)

    while not finished:
        # Add basis and reinitialize model if needed
        # model.H_size_added += basis_added_step
        model.reinitialize()
        finished = fitting(model, optimizer, loss_threshold, max_training_steps, references, kvectors)

    if finished:
        Resulting_Hamiltonian = model.HR.numpy()
        Rvectors_of_the_resulting_hamiltonian = model.R.numpy().astype(np.int32)
        Reproduced_TB_bandstructure = model.wholebandstructure.numpy()
        Reproduced_TB_bands = model.reproduces.numpy()

        np.save("./data/output/InSe Nanoribbon/resulting_real_space_hamiltonians.npy", Resulting_Hamiltonian)
        np.save("./data/output/InSe Nanoribbon/rvectors_of_the_resulting_hamiltonian.npy", Rvectors_of_the_resulting_hamiltonian)
        np.save("./data/output/InSe Nanoribbon/TB_bandstructure.npy", Reproduced_TB_bandstructure)
        np.save("./data/output/InSe Nanoribbon/reproduced_TB_bands.npy", Reproduced_TB_bands)

        # Calculate and print the final loss value
        final_loss_value = tf.reduce_mean(tf.square(model.reproduces - model.references)).numpy()
        print(f"final loss value: {final_loss_value:.8f}")
    

if __name__ == '__main__':
    main()

TypeError: 'NoneType' object is not subscriptable